In [4]:
from flask import Flask, request, send_file
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd

def caputrar_esportedasorte():
    # Especifica o caminho do arquivo que deseja excluir
    caminho_arquivo = "partidas.csv"

    # Verifica se o arquivo existe antes de excluí-lo
    if os.path.exists(caminho_arquivo):
        os.remove(caminho_arquivo)
        print(f"O arquivo {caminho_arquivo} foi excluído com sucesso")
    else:
        print(f"O arquivo {caminho_arquivo} não existe")

    chrome_options = Options()                   
    chrome_options.add_argument("--user-data-dir=C:\\Users\\mauri\\AppData\\Local\\Google\\Chrome\\User Data\\Default") 
    driver = webdriver.Chrome(options=chrome_options)

    # acessar o site
    driver.get("https://m.esportesdasorte.com/ptb")
    sleep(4)        
    try: 

        # acessar icone do usuario
        try:
            icon_user = driver.find_element(by=By.XPATH, value='//*[@id="hdr-c"]/nav/div/a[3]/i')
            print(icon_user)
            icon_user.click()
            sleep(1)

            # acessar o login
            button_login = driver.find_element(by=By.XPATH, value='//*[@id="user-menu"]/div[2]/div[1]/a[1]')
            button_login.click()
            sleep(1)

            # preencher login, senha e entrar
            input_user = driver.find_element(by=By.XPATH, value='//*[@id="username"]')
            input_user.send_keys('mauriciopfleger')
            input_pass = driver.find_element(by=By.XPATH, value='//*[@id="password"]')
            input_pass.send_keys('Jaqueta1!@')
            button_entrar = driver.find_element(by=By.XPATH, value='//*[@id="allCnt"]/main/app-authentication/app-signin/div/form/button')
            button_entrar.click()
            sleep(2)
            print('realizado login')
        except:
            print('logado')

        lista_campeonato = ['brasileirao', 'copadobrasil', 'libertadores']

        registros_resultado = []
        registros_dupla_chance = []

        for campeonato in lista_campeonato:
            try:
                if (campeonato == 'brasileirao'):
                    # acessar o site
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/brazil")
                    sleep(4)    
                elif (campeonato == 'copadobrasil'):
                    # acessar o site
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/brazil/copa-do-brasil-2024")
                    sleep(4)    
                elif (campeonato == 'libertadores'):
                    # acessar o site
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/international-clubs")
                    sleep(4)   

                for i in range(2):
                    if i == 1: 
                        if (campeonato == 'brasileirao'):
                            dupla_chance = driver.find_element(by=By.XPATH, value='//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list/ul/li[5]/a')

                        elif (campeonato == 'copadobrasil'):
                            dupla_chance = driver.find_element(by=By.XPATH, value='//*[@id="allCnt"]/main/place-bet/div/sports/main/app-btg-list/ul/li[5]/a')

                        elif (campeonato == 'libertadores'):
                            dupla_chance = driver.find_element(by=By.XPATH, value='//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list[1]/ul/li[5]/a')
                            
                        dupla_chance.click()
                        sleep(1)

                    divs_resultado = driver.find_elements(by=By.CLASS_NAME, value='match-content')
                    for div in divs_resultado:
                        # div.text = 'Vitória BA \n Goianiense \n 18/05 16:00 \n BB \n +354 \n equalizer \n 1.94 \n 3.34 \n 4.45'
                        infos = div.text.split('\n')
                        time_casa = infos[0]
                        time_visitante = infos[1]
                        data = infos[2]

                        odd_casa = ""
                        odd_empate = ""
                        odd_fora = ""
                        odd_1x = ""
                        odd_12 = ""
                        odd_x2 = ""

                        if i == 0:
                            odd_casa = infos[len(infos)-3]
                            odd_empate = infos[len(infos)-2]
                            odd_fora = infos[len(infos)-1]
                            registros_resultado.append([campeonato, time_casa, time_visitante, data, odd_casa, odd_empate, odd_fora])
                        else: 
                            odd_1x = infos[len(infos)-3]
                            odd_12 = infos[len(infos)-2]
                            odd_x2 = infos[len(infos)-1]
                            registros_dupla_chance.append([campeonato, time_casa, time_visitante, data, odd_1x, odd_12, odd_x2])
            except:
                continue

        driver.quit() 
                
        df_resultado = pd.DataFrame(registros_resultado, columns=['campeonato', 'time_casa', 'time_visitante', 'data', 'odd_casa', 'odd_empate', 'odd_visitante'])
        df_dupla_chance = pd.DataFrame(registros_dupla_chance, columns=['campeonato', 'time_casa', 'time_visitante', 'data', 'odd_1x', 'odd_12', 'odd_x2'])
        new_df = pd.concat([df_resultado, df_dupla_chance[['odd_1x', 'odd_12', 'odd_x2']]], axis=1)
        new_df.to_csv(caminho_arquivo, index=False, sep=';', encoding='utf-8')

        print('capturado')
    except: 
        driver.quit()

def apostar_esportedasorte():
    file = 'bilhetes.csv'
    df = pd.read_csv(file, delimiter=';')
    df.set_index('Numero', inplace=True)

    chrome_options = Options()                   
    chrome_options.add_argument("--user-data-dir=C:\\Users\\mauri\\AppData\\Local\\Google\\Chrome\\User Data\\Default") 
    driver = webdriver.Chrome(options=chrome_options)

    # acessar o site
    driver.get("https://m.esportesdasorte.com/ptb")
    sleep(10)    
    try:

        # acessar icone do usuario
        try:
            icon_user = driver.find_element(by=By.XPATH, value='//*[@id="hdr-c"]/nav/div/a[3]/i')
            print(icon_user)
            icon_user.click()
            sleep(1)

            # acessar o login
            button_login = driver.find_element(by=By.XPATH, value='//*[@id="user-menu"]/div[2]/div[1]/a[1]')
            button_login.click()
            sleep(1)

            # preencher login, senha e entrar
            input_user = driver.find_element(by=By.XPATH, value='//*[@id="username"]')
            input_user.send_keys('mauriciopfleger')
            input_pass = driver.find_element(by=By.XPATH, value='//*[@id="password"]')
            input_pass.send_keys('Jaqueta1!@')
            button_entrar = driver.find_element(by=By.XPATH, value='//*[@id="allCnt"]/main/app-authentication/app-signin/div/form/button')
            button_entrar.click()
            sleep(2)
            print('realizado login')
        except:
            print('logado')

        try:
            apostas = driver.find_element(by=By.XPATH, value='//*[@id="bslpb"]/span/small')
            if (int(apostas.text) > 0):
                apostas.click()
                sleep(1)
                apagar = driver.find_element(by=By.XPATH, value='//*[@id="multi-betslip"]/div[2]/div[3]/div[3]/a[1]')
                apagar.click()
                sleep(1)
        except:
            print('sem apostas')

        previous_campeonato = None
        previous_index = 0
        valor_aposta = '0'
        qtd_jogos = 0
        skipou = False
        for index, row in df.iterrows():
            if row['Apostado'] == 1:
                previous_index = index
                skipou = True
                continue

            qtd_jogos += 1
            if (index != previous_index) and (not skipou):
                sleep(1)
                apostas = driver.find_element(by=By.XPATH, value='//*[@id="bslpb"]/span/small')
                apostas.click()
                sleep(1)       

                # apagar = driver.find_element(by=By.XPATH, value='//*[@id="multi-betslip"]/div[2]/div[3]/div[3]/a[1]')
                # apagar.click()
                # sleep(1)

                if qtd_jogos == 1:                                  
                    valor = driver.find_element(by=By.XPATH, value='//*[@id="single-betslip"]/div[1]/app-coupon/div/div[2]/div[2]')
                else:                                               
                    valor = driver.find_element(by=By.XPATH, value='//*[@id="multi-betslip"]/div[2]/div[3]/div[1]/div/div/div[2]')
                valor.click()
                sleep(1)      

                qtd_jogos = 0  

                for num in valor_aposta:
                    vnum = num
                    if num == '0':
                        vnum = '10'
                    valor_num = driver.find_element(by=By.XPATH, value=f'//*[@id="cNumpad"]/app-custom-numpad/div[3]/button[{vnum}]')
                    valor_num.click()
                    sleep(1)
                
                enter = driver.find_element(by=By.XPATH, value='//*[@id="cNumpad"]/app-custom-numpad/div[3]/button[13]')
                enter.click()
                sleep(3)
                try:
                    aceitar = driver.find_element(by=By.XPATH, value='//*[@id="multi-betslip"]/div[2]/div[2]/div[2]/a')
                    if (aceitar.text.strip() == 'Aceitar todas as alterações'):
                        aceitar.click()
                        sleep(3)
                except:
                    print('direto')

                apostar = driver.find_element(by=By.XPATH, value='//*[@id="betslip-placebet-btn"]')
                apostar.click()
                sleep(3)
                voltar = driver.find_element(by=By.XPATH, value='//*[@id="preview-betslip"]/div[3]/a[1]')
                voltar.click()
                sleep(1)

            previous_index = index
            
            skipou = False
            
            valor_aposta = str(row['Aposta'])
            
            if (row['Campeonato'] == 'brasileirao'):
                # acessar o site
                if previous_campeonato != row['Campeonato']:
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/brazil")
                    sleep(7)    
                else: 
                    resultado = driver.find_element(by=By.XPATH, value='//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list/ul/li[1]/a')
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", resultado)
                    driver.execute_script("window.scrollBy(0, -100);")
                    resultado.click()
                    sleep(1)
            elif (row['Campeonato'] == 'copadobrasil'):
                # acessar o site
                if previous_campeonato != row['Campeonato']:
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/brazil/copa-do-brasil-2024")
                    sleep(7)    
                else: 
                    resultado = driver.find_element(by=By.XPATH, value='//*[@id="allCnt"]/main/place-bet/div/sports/main/app-btg-list/ul/li[1]/a')
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", resultado)
                    driver.execute_script("window.scrollBy(0, -100);")
                    resultado.click()
                    sleep(1)    
            elif (row['Campeonato'] == 'libertadores'):
                # acessar o site
                if previous_campeonato != row['Campeonato']:
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/international-clubs")
                    sleep(7)    
                else: 
                    resultado = driver.find_element(by=By.XPATH, value='//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list[1]/ul/li[1]/a')
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", resultado)
                    driver.execute_script("window.scrollBy(0, -100);")
                    resultado.click()
                    sleep(1)   
            
            previous_campeonato = row['Campeonato']

            for i in range(2):
                if i == 1: 
                    if (row['Campeonato'] == 'brasileirao'):
                        dupla_chance = driver.find_element(by=By.XPATH, value='//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list/ul/li[5]/a')

                    elif (row['Campeonato'] == 'copadobrasil'):
                        dupla_chance = driver.find_element(by=By.XPATH, value='//*[@id="allCnt"]/main/place-bet/div/sports/main/app-btg-list/ul/li[5]/a')

                    elif (row['Campeonato'] == 'libertadores'):
                        dupla_chance = driver.find_element(by=By.XPATH, value='//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list[1]/ul/li[5]/a')

                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", dupla_chance)   
                    dupla_chance.click()
                    sleep(1)

                parar = False
                divs_resultado = driver.find_elements(by=By.CLASS_NAME, value='match-content')
                for div in divs_resultado:
                    # div.text = 'Vitória BA \n Goianiense \n 18/05 16:00 \n BB \n +354 \n equalizer \n 1.94 \n 3.34 \n 4.45'
                    infos = div.text.split('\n')
                    time_casa = infos[0]
                    time_visitante = infos[1]
                    data = infos[2]

                    jogo = str.split(row['Jogo'], ' x ')
                    dftime_casa = jogo[0]
                    dftime_visitante = jogo[1]
                    dfdata = row['Data']

                    if (time_casa == dftime_casa and
                        time_visitante == dftime_visitante and
                        data == dfdata):
                        elementos_internos = div.find_elements(By.TAG_NAME, 'a')  
                        if (i == 0 and pd.isna(row['Resultado2'])):
                            if (row['Resultado1'] == time_casa):                            
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[1])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[1].click()
                                parar = True
                                break
                            elif (row['Resultado1'] == time_visitante): 
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[3])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[3].click()
                                parar = True
                                break
                            elif (row['Resultado1'] == 'Empate'):
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[2])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[2].click()
                                parar = True
                                break
                        elif (i == 1 and not pd.isna(row['Resultado2'])):
                            if (row['Resultado1'] == time_casa and row['Resultado2'] == 'Empate'):
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[1])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[1].click()
                                break
                            elif (row['Resultado1'] == 'Empate' and row['Resultado2'] == time_visitante):
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[3]) 
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[3].click()
                                break
                            elif (row['Resultado1'] == time_casa and row['Resultado2'] == time_visitante):
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[2])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[2].click()
                                break

                if parar:
                    break
                
        sleep(1)        
        apostas = driver.find_element(by=By.XPATH, value='//*[@id="bslpb"]/span/small')
        apostas.click()
        sleep(1)

        # apagar = driver.find_element(by=By.XPATH, value='//*[@id="multi-betslip"]/div[2]/div[3]/div[3]/a[1]')
        # apagar.click()
        # sleep(1)

        if qtd_jogos == 1:                                  
            valor = driver.find_element(by=By.XPATH, value='//*[@id="single-betslip"]/div[1]/app-coupon/div/div[2]/div[2]')
        else:                                               
            valor = driver.find_element(by=By.XPATH, value='//*[@id="multi-betslip"]/div[2]/div[3]/div[1]/div/div/div[2]')
        valor.click()
        sleep(1)      

        qtd_jogos = 0  

        for num in valor_aposta:
            vnum = num
            if num == '0':
                vnum = '10'
            valor_num = driver.find_element(by=By.XPATH, value=f'//*[@id="cNumpad"]/app-custom-numpad/div[3]/button[{vnum}]')
            valor_num.click()
            sleep(1)
        
        enter = driver.find_element(by=By.XPATH, value='//*[@id="cNumpad"]/app-custom-numpad/div[3]/button[13]')
        enter.click()
        sleep(3)
        try:
            aceitar = driver.find_element(by=By.XPATH, value='//*[@id="multi-betslip"]/div[2]/div[2]/div[2]/a')
            if (aceitar.text.strip() == 'Aceitar todas as alterações'):
                aceitar.click()
                sleep(3)
        except:
            print('direto')

        apostar = driver.find_element(by=By.XPATH, value='//*[@id="betslip-placebet-btn"]')
        apostar.click()
        sleep(3)
        voltar = driver.find_element(by=By.XPATH, value='//*[@id="preview-betslip"]/div[3]/a[1]')
        voltar.click()
        sleep(1)

        driver.quit() 

        print('apostado')
    except:
        driver.quit() 

#apostar_esportedasorte()

app = Flask(__name__)

@app.route('/buscar')
def run_buscar():

    caputrar_esportedasorte()
    
    caminho_arquivo = "partidas.csv"

    return send_file(
        caminho_arquivo,
        mimetype='text/csv',
        as_attachment=True,
        download_name=caminho_arquivo
    )

@app.route('/apostar', methods=['POST'])
def run_apostar():
    if 'file' not in request.files:
        return "No file part in the request", 400
    file = request.files['file']
    if file.filename == '':
        return "No file selected", 400
    
    # Especifica o caminho do arquivo que deseja excluir
    caminho_arquivo = "bilhetes.csv"

    # Verifica se o arquivo existe antes de excluí-lo
    if os.path.exists(caminho_arquivo):
        os.remove(caminho_arquivo)
        print(f"O arquivo {caminho_arquivo} foi excluído com sucesso")
    else:
        print(f"O arquivo {caminho_arquivo} não existe")

    df = pd.read_csv(file, delimiter=';')
    df.to_csv(caminho_arquivo, sep=';', index=False)  # Salva o DataFrame sem a coluna de índice

    apostar_esportedasorte()

    return "Apostas realizadas!"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5032, debug=False)

KeyboardInterrupt: 